In [1]:
import pandas as pd
import numpy as np


def get_shrunk_covariance_matrix(x, shrink=None):
        """Computes a covariance matrix that is "shrunk" towards a structured estimator. Code
            borrows heavily from the MATLAB implementation available by the authors online.
        
        Parameters
        ----------
        x : N x N sample covariance matrix of stock returns
        shrink : given shrinkage intensity factor; if none, code calculates
        
        Returns
        -------
        tuple : pandas.DataFrame which contains the shrunk covariance matrix
                : float shrinkage intensity factor
        
        """
        if x is None:
            raise ValueError('No covariance matrix defined')

        if type(x) == pd.core.frame.DataFrame:
            cov = x.as_matrix()
        elif type(x) == np.ndarray:
            cov = x
        else:
            raise ValueError(
                'Covariance matrix passed must be numpy.ndarray or pandas.DataFrame')

        if shrink is not None:
            shrinkage = shrink

        index = x.index
        columns = x.columns

        [t, n] = np.shape(cov)
        meanx = cov.mean(axis=0)
        cov = cov - np.tile(meanx, (t, 1))

        sample = (1.0 / t) * np.dot(cov.T, cov)

        var = np.diag(sample)
        sqrtvar = np.sqrt(var)

        a = np.tile(sqrtvar, (n, 1))
        rho = (sum(sum(sample / (a * a.T))) - n) / (n * (n - 1))

        prior = rho * (a * a.T)
        prior[np.eye(t, n) == 1] = var

        # Frobenius-norm of matrix cov, sqrt(sum(diag(dot(cov.T, cov))))
        # have to research this
        c = np.linalg.norm(sample - prior, 'fro') ** 2
        y = cov ** 2.0
        p = np.dot((1.0 / t), sum(sum(np.dot(y.T, y)))) - \
            sum(sum(sample ** 2.0))
        rdiag = np.dot((1.0 / t), sum(sum(y ** 2.0))) - sum(var ** 2.0)
        v = np.dot((cov ** 3.0).T, cov) / t - ((var * sample).T)
        v[np.eye(t, n) == 1] = 0.0
        roff = sum(sum(v * (a / a.T)))
        r = rdiag + np.dot(rho, roff)

        # compute shrinkage constant
        k = (p - r) / c
        shrinkage = max(0.0, min(1.0, k / t))
        sigma = np.dot(shrinkage, prior) + np.dot((1 - shrinkage), sample)

        return pd.DataFrame(sigma, index=index, columns=columns), shrinkage

df = pd.read_csv('../input/stocks_JT.csv')

def c_date(x):
    d, m, y = x.split('/')
    return '{}-{}-{}'.format(y, m, d)

df.date = pd.to_datetime(df.date.apply(c_date))
df = df.sort_values('date')

In [2]:
%%time

cov_matrices = []

start_year = 1988
time_window = 10 

for start_year in range(1988, 2018 + 10, 10):
    
    print("Filtering year ", start_year)

    df_hist = df[df['date'].dt.year <= start_year]
    df_hist['size'] = df_hist.groupby('TICKER')['date'].transform('count')
    df_hist = df_hist[df_hist['size'] >= 11 * 12]

    df_hist['ret'] = df_hist.groupby('TICKER')['PRC'].transform(pd.Series.pct_change)

    ret_df = df_hist.groupby('TICKER')['ret'].apply(lambda x: list(x)[-120:])

    ret_df = pd.DataFrame(dict(ret_df))

    cov_matrices.append(ret_df)
    

Filtering year  1988


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Filtering year  1998
Filtering year  2008
Filtering year  2018
CPU times: user 26.7 s, sys: 1.35 s, total: 28 s
Wall time: 27.7 s


In [3]:
cov_matrices[0]

,ACV,AMCO,AZE,BFD,BIO,BLT,BSC,BTI,CHL,CIS,CITU,CMD,CNP,CPN,DEP,EJ,FCE,HUB,KC,MOG,MRI,OHC,PDL,PLR,RA,RT,SAU,SK,SMC,TDD,UFD,VGC,WAN,WIN,WSO
0,-0.092784,-0.224138,-2.080460,-0.047445,0.000000,-0.201613,-0.006410,-0.034884,-0.033537,-1.980000,-2.005000,-0.027322,0.253968,0.000000,-1.964000,-2.080000,-1.878289,-2.081395,0.000000,-0.157143,-0.093750,0.092593,-0.219178,0.050000,-0.142857,0.004739,-1.961538,-0.025510,-0.134615,-0.266667,-0.040000,-0.075758,-0.164912,-0.024272,-2.000000
1,0.000000,0.066667,-0.021277,0.034483,-0.014815,0.000000,-0.006452,-2.012048,0.037855,-1.857143,-2.084577,-0.016854,-0.139241,0.090909,-1.954357,-2.000000,0.000000,-0.060932,0.066667,0.008475,0.344828,-0.016949,-2.280702,0.047619,0.055556,-0.087264,-1.986667,0.335079,0.096296,0.018182,0.000000,0.000000,-0.004202,0.402985,0.009804
2,0.136364,-0.187500,-1.916667,0.066667,-0.097744,-0.090909,-0.012987,-2.011905,-0.042553,-0.095238,-0.110092,-0.097143,0.125000,-0.027778,0.008696,-0.012346,-0.089888,-0.091603,-0.062500,-0.151261,-0.358974,0.000000,-1.808219,-0.090909,-2.131579,-0.100775,0.013514,-0.309804,-0.135135,0.089286,0.000000,-0.163934,-0.042194,-2.049645,0.048544
3,0.080000,0.025641,-2.011858,-0.062500,-0.008333,0.011111,0.144737,-0.011765,0.082540,0.105263,0.206186,-0.006329,-0.183007,0.000000,0.090517,-2.000000,0.008230,0.012605,-0.183333,0.000000,-2.060000,-0.034483,-2.271186,-1.975000,-2.069767,0.103448,-2.026667,0.448864,-0.132812,-2.024590,0.000000,0.372549,-0.022026,-1.709459,-2.023148
4,-0.064815,0.200000,-1.968750,0.200000,-0.016807,0.021978,0.005747,0.035714,0.076246,0.047619,-0.256410,-0.076433,0.120000,0.000000,0.162055,-2.150000,-2.020408,-0.004149,-0.102041,0.039604,-2.132075,-0.017857,0.206667,0.076923,0.304348,-0.067708,0.025974,-0.294118,0.144144,-1.944000,0.041667,-0.171429,-0.063063,0.485714,-2.004525
5,0.108911,-0.166667,-2.016129,-0.074074,0.008547,-0.021505,0.022857,0.080460,0.100817,-0.045455,0.264368,0.000000,0.221429,-0.071429,-2.023810,0.000000,-1.988000,-0.029167,0.090909,0.038095,-0.033333,-0.018182,-1.696133,-2.000000,0.000000,-2.103352,0.000000,0.377778,0.133858,0.033898,-0.040000,0.017241,0.019231,-0.285256,0.099099
6,-1.941964,0.000000,-0.063492,-0.016667,-0.008475,0.186813,-0.005587,-0.010638,0.051980,-0.333333,-0.136364,0.186207,-0.058480,0.076923,-0.028239,-1.956522,0.040486,0.047210,-0.041667,-0.027523,-0.206897,0.018519,-2.380952,-1.952381,0.000000,-1.815190,-2.215190,-0.008065,-0.069444,-1.918033,0.125000,0.000000,-2.066038,-2.524664,-0.032787
7,-1.976303,0.525000,-2.025424,-0.023729,-1.987179,0.000000,-0.117978,0.032258,0.025882,0.071429,0.178947,0.011628,-1.819876,0.000000,-0.063248,-2.000000,-0.035019,-0.012295,-0.108696,0.000000,0.086957,0.036364,-0.304598,-2.100000,-0.033333,0.111801,-0.145833,-0.349593,-1.962687,0.035714,0.000000,-0.186441,-2.004425,-1.679412,0.050847
8,0.048544,0.000000,-0.037190,-0.041667,0.008658,-0.092593,-0.006369,-0.093750,0.055046,-2.033333,-0.142857,0.051724,0.098485,0.021429,-2.000000,-1.914773,-1.975806,-0.008299,0.024390,-0.056604,-1.800000,-1.947368,-0.033058,-2.113636,-0.034483,-0.114525,-0.195122,-0.162500,-1.837209,-1.931034,-0.111111,-0.125000,-0.118943,0.012987,0.024194
9,-0.120370,-0.409836,-2.030043,0.039855,-0.004292,-2.015306,-0.128205,0.000000,0.004348,0.000000,0.156250,-0.027322,-1.841379,0.000000,0.018248,0.000000,0.049587,0.041841,-0.095238,-0.010000,-2.150000,-0.018519,0.487179,-0.020408,0.000000,0.063091,-0.121212,0.925373,0.046296,0.018519,-0.083333,-0.095238,-0.025000,-2.478632,-0.047244


In [4]:
cov_matrices[1]

,A,AA,AAA,AAC,AAGIY,AAL,AALA,AAME,AAPL,AATI,ABBK,ABC,ABCC,ABEV,ABF,ABG,ABIG,ABIO,ABL,ABM,ABP,ABRI,ABS,ABSI,ABT,ABW,ABX,ABY,AC,ACAD,ACB,ACC,ACCC,ACCMA,ACCMB,ACCOB,ACD,ACE,ACES,ACET,...,WWY,WX,WY,WYL,WYMN,X,XETA,XICO,XIOX,XOMA,XON,XPLR,XRAY,XRIT,XRX,XTR,XTX,Y,YELL,YFED,YLD,YOCM,YORK,Z,ZAPS,ZCAD,ZE,ZEGL,ZEUS,ZF,ZIGO,ZION,ZITL,ZMX,ZNIC,ZNT,ZOS,ZRN,ZRO,ZSEV
0,-0.096774,0.154018,0.006211,-0.428571,-0.045872,0.127273,-0.285701,-0.074074,-0.062112,0.000000,0.049180,-0.136240,-0.051282,-0.092199,0.097701,-0.091837,0.018692,0.131868,-1.964286,0.072398,0.057143,-0.022727,0.108911,-2.060606,0.018182,-0.125000,0.160305,0.061538,0.083333,0.100840,0.036145,-0.062500,-0.125000,0.216418,0.210526,-0.056250,-0.062500,0.080000,-0.200000,-0.032258,...,0.086505,0.146974,0.074627,-0.031447,-0.033333,0.068493,-0.473684,-0.079365,0.391304,0.103774,0.053977,-2.090909,-0.032258,0.000000,0.068522,0.123333,0.013158,0.116197,-0.019763,-0.057971,-0.038961,0.000000,0.000000,0.045894,0.133333,-0.319149,0.158730,-0.076923,0.033333,0.024096,-2.054054,-0.034091,0.523810,-0.087379,-0.333333,-0.007143,-0.177778,0.137725,0.040650,-0.107692
1,0.107143,-0.048356,-0.425926,-0.104167,0.081731,-0.153226,0.000000,0.000000,-0.039735,0.016949,0.093750,-0.066246,-0.162162,-0.023438,-0.052356,-0.056180,-0.045872,-0.048544,-1.976431,-0.016878,0.018018,-2.046512,-0.050595,0.092857,0.043367,-0.242857,0.138158,-0.021739,0.043956,0.229008,0.000000,-0.133333,0.095238,0.055215,0.043478,-0.019868,0.000000,-0.074074,1.625040,0.008333,...,-0.050955,0.010050,-0.041667,-0.396104,0.000000,0.089744,0.250000,0.017241,0.000000,-0.025641,-0.053908,0.416667,-1.975000,0.166667,-0.012024,-0.097923,-0.012987,0.033123,-0.060484,-0.015385,-0.054054,0.107143,0.000000,-0.034642,-0.205882,-0.281250,-0.155251,-0.033333,-0.129032,-0.035294,0.000000,0.094118,-0.125000,-1.952128,-0.071429,0.057554,-0.135135,-0.052632,0.039062,0.017241
2,-0.096774,-0.026423,-0.064516,0.186047,-0.017778,-0.057143,0.000000,-0.040000,-0.017241,-0.041667,-1.914286,0.060811,-0.064516,0.056000,0.149171,0.119048,-0.048077,-0.071429,-2.027586,0.154506,0.030973,0.000000,0.043887,-2.071895,0.070905,0.075472,-0.028902,-0.029630,-0.010526,-0.329193,-0.011628,-1.884615,0.000000,0.023256,0.041667,-0.006757,0.000000,0.140000,-0.333343,0.024793,...,0.020134,0.059701,0.028986,-0.118280,0.017241,-0.015686,0.200000,0.000000,-0.093750,0.192982,0.000000,0.176471,-1.923077,-0.119048,-0.040568,-0.013158,-1.967105,0.004580,-0.068670,-0.062500,-0.014286,-0.032258,-0.020833,-0.040670,-0.074074,0.260870,-0.081081,-0.017241,-2.092593,0.048780,0.025641,-0.021505,0.053571,-2.005587,0.307692,-0.006803,0.031250,0.150000,-0.037594,-2.016949
3,-0.107143,0.056367,-0.333333,0.156863,-0.027149,0.045455,0.000000,0.000000,0.094737,0.026087,-2.031250,-0.082803,0.413793,0.007576,-0.048077,-0.031915,-0.242424,0.131868,-1.986577,0.026022,-0.012876,0.044444,0.066066,-0.121951,0.009132,0.000000,-0.023810,0.030534,0.063830,0.268519,-0.047059,-0.086957,-0.043478,0.153409,0.154286,-0.027211,-0.040000,0.035088,0.071451,-0.016129,...,0.184211,-0.028169,0.098592,0.195122,0.050847,0.039841,-0.066667,0.067797,-0.034483,-0.073529,-0.017094,-0.100000,0.037037,0.121622,0.141649,0.060000,-2.020408,0.063830,-0.023041,0.058333,-0.014493,-0.066667,0.127660,0.052369,-0.120000,0.086207,-0.017647,0.122807,-0.101695,0.011628,0.000000,0.126374,0.220339,-1.972222,0.029412,0.075342,0.060606,0.009662,0.101562,-0.025000
4,-0.320000,0.023715,0.043103,0.067797,-0.060465,-0.111111,0.000000,0.125000,0.224359,0.000000,0.166667,-0.131944,0.121951,0.030075,0.055556,-0.021978,-0.200000,0.097087,-2.006803,0.061594,0.113043,0.021277,0.028169,0.000000,0.072398,0.228070,-0.036585,-0.007407,-0.030000,-0.058394,-0.006173,0.000000,-1.977273,0.078818,0.084158,0.034965,0.027778,0.084746,-0.133330,0.000000,...,0.061111,0.007246,0.017094,-0.183673,0.032258,0.053640,0.071429,0.015873,-0.142857,0.126984,0.000000,-0.222222,0.017857,-0.012048,-0.055556,-0.097484,0.013333,-0.040000,0.056604,0.007874,-0.014706,0.4

In [5]:
cov_matrices[2]

,A,AA,AAA,AAC,AACE,AAGIY,AAL,AALA,AAME,AANB,AAON,AAP,AAPL,AATI,AB,ABAN,ABAX,ABBK,ABC,ABCB,ABCC,ABCO,ABCW,ABD,ABERF,ABEV,ABF,ABFS,ABG,ABI,ABIG,ABIO,ABIX,ABK,ABL,ABM,ABMC,ABMD,ABN,ABP,...,YFED,YHOO,YLD,YLH,YLT,YOCM,YORK,YPF,YRK,YUM,Z,ZAP,ZAPS,ZBRA,ZCAD,ZCO,ZE,ZEGL,ZEUS,ZF,ZIF,ZIGO,ZILA,ZION,ZITL,ZLC,ZMX,ZNH,ZNIC,ZNT,ZOLL,ZOLT,ZONA,ZOOM,ZOS,ZRAN,ZRN,ZRO,ZSEV,ZTR
0,-0.067073,0.120704,0.000000,0.000000,-1.995968,0.108911,0.127273,-2.333191,-0.102564,-0.260870,0.020134,-0.118911,0.006107,0.022727,0.066975,-0.085470,-0.193548,0.187500,0.142857,-0.039604,-0.051282,0.212209,-0.221354,-0.313953,-0.250000,-0.092199,0.021390,0.443850,-0.256757,0.002083,0.018692,-0.288889,-2.162791,-0.006231,0.243902,-0.045126,0.194805,-0.121019,-0.209677,0.137500,...,-1.857143,0.495120,0.044444,0.036364,0.040000,-2.166667,-0.215686,0.140940,-0.023936,-0.051122,0.094828,-0.086735,0.280000,0.050000,-0.319149,0.091603,0.158730,-0.076923,0.475000,-0.005747,0.039604,0.275362,-0.509494,-0.070140,0.230769,0.017442,0.104167,-0.054795,-0.333333,0.008108,0.239437,-0.013605,-0.170455,0.187500,-0.177778,0.132143,0.137725,0.040650,-1.986301,-0.021127
1,-0.156863,-0.515333,-0.617188,0.258621,-1.728745,0.017857,-0.153226,-1.750080,-0.042857,-0.058824,-0.098684,-0.138211,-0.154780,0.000000,-0.021645,-0.224299,0.080000,0.087719,-0.017857,0.000000,-0.162162,0.016787,-0.073579,-0.135593,0.111111,-0.023438,0.261780,-0.088889,0.145455,0.003119,-0.045872,-0.062500,-0.200000,-0.063741,0.009804,-0.132325,-0.195652,0.057971,0.122449,-0.027473,...,-2.083333,-0.566690,0.000000,-0.035088,0.000000,-2.142857,-0.175000,-0.090196,0.043597,0.306176,-0.043307,-0.156425,-0.031250,-0.144928,-0.281250,-0.027972,-0.155251,-0.033333,0.135593,-0.057803,0.009524,0.022727,-0.154839,0.103448,0.041667,0.009524,0.113208,-0.043478,-0.071429,-0.016086,0.056818,-0.051724,-0.054795,-0.131579,-0.135135,-0.091483,-0.052632,0.039062,-0.013889,-0.050360
2,0.065891,0.016975,0.918367,-0.164384,0.066667,0.000000,-0.057143,-2.333191,-0.044776,-0.098958,0.284672,-0.115094,0.032316,-0.122222,-0.172566,0.072289,0.148148,-0.064516,-0.654545,0.005155,-0.064516,-0.208726,-0.111913,0.058824,0.100000,0.056000,0.095436,-0.113821,-0.253968,-0.139896,-0.048077,0.133333,0.050000,-0.035714,-0.271845,0.063181,-0.135135,0.369863,-0.454545,0.045198,...,-0.076923,0.096906,0.042553,0.036364,0.019231,0.312500,0.696970,0.088362,0.023499,0.130785,-0.032922,-0.052980,0.096774,-0.079903,0.260870,0.035971,-0.081081,-0.017241,-0.164179,0.036810,-0.094340,0.055556,-0.076336,0.039062,-0.060000,0.032075,0.033898,0.272727,0.307692,0.059946,-0.053763,-0.127273,-0.173913,-0.060606,0.031250,-0.090278,0.150000,-0.037594,-0.042254,0.000000
3,-0.043636,0.511381,-0.574468,0.245902,0.072917,0.100877,0.045455,-2.000000,-0.015625,-0.005780,0.056818,-0.189765,0.280000,0.088608,0.069519,0.134831,0.032258,0.168103,-0.263158,0.179487,0.413793,0.025335,0.195122,0.185185,1.090909,0.007576,0.064394,0.330275,0.170213,-0.180723,-0.242424,0.352941,-2.142857,0.118056,0.026667,-0.004098,-0.062500,0.360000,-0.233333,-0.037838,...,-2.236111,0.037491,0.000000,0.000000,-0.037736,-1.952381,0.017857,0.390099,-0.020408,-0.083630,-0.076596,-0.013986,-0.117647,0.400000,0.086207,-0.062500,-0.017647,0.122807,0.187500,0.000000,0.000000,-0.178947,0.057851,0.002820,-0.212766,0.106033,-0.016393,0.488095,0.029412,-0.023136,0.102273,-0.075000,0.070175,0.008065,0.060606,-0.339695,0.009662,0.101562,-0.051471,0.007576
4,0.125475,-0.116466,-0.275000,0.434211,-0.208738,0.035857,-0.111111,-2.000000,0.111111,-0.017442,-0.091398,-0.150000,-0.042120,-0.023256,0.157500,0.039604,0.093750,-0.099631,-0.607143,-0.013043,0.121951,0.110465,-0.102041,0.015625,0.434783,0.030075,-0.007117,-0.020690,-0.436364,0.051471,-0.200000,0.021739,-2.041667,-0.034161,0.064935,-0.100823,-0.133333,-0.058824,0.000000,-0.078652,...,0.101124,-0.152773,0.020408,0.052632,0.019608,-1.850000,0.035088,-0.039886,0.101562,-0.095146,-0.082949,-0.085106,-1.983333,-0.033835,-0.015873,-1.988889,-0.065868,0.125000,-0.082707,

In [6]:
cov_matrices[3]

,A,AA,AAA,AAC,AACE,AAGIY,AAI,AAL,AALA,AAME,AAN,AANB,AAON,AAP,AAPL,AAT,AATI,AAU,AAV,AAWW,AB,ABAN,ABAX,ABB,ABBK,ABC,ABCB,ABCC,ABCO,ABCW,ABD,ABERF,ABEV,ABF,ABFS,ABG,ABI,ABIG,ABIO,ABIX,...,YZC,Z,ZAP,ZAPS,ZBRA,ZCAD,ZCO,ZE,ZEGL,ZEN,ZEUS,ZF,ZGEN,ZHNE,ZIF,ZIGO,ZILA,ZION,ZIOP,ZITL,ZIXI,ZLC,ZMH,ZMX,ZN,ZNH,ZNIC,ZNT,ZOLL,ZOLT,ZONA,ZOOM,ZOS,ZQK,ZRAN,ZRN,ZRO,ZSEV,ZTR,ZUMZ
0,-0.036051,-0.048000,0.000000,-0.159091,-1.995968,0.108911,-0.047398,0.019608,-2.333191,-1.996667,0.286595,-0.021277,0.123619,0.047457,0.066562,0.013298,-0.043478,-0.295337,0.049074,0.199668,0.043888,-0.085470,-0.001005,-0.062929,0.187500,-0.020540,0.120224,-0.051282,-0.012613,-0.006211,0.174603,-0.250000,-0.050000,0.021390,0.153223,0.225025,0.002083,0.018692,-0.736842,-2.162791,...,0.041820,0.328671,-0.149425,0.280000,0.013307,-0.319149,0.091603,0.158730,-0.076923,0.004444,-0.063717,-0.006912,0.000000,-0.339506,0.039604,-0.134048,-0.037736,-0.083020,-0.078431,0.230769,0.000000,-0.500458,-0.005354,0.104167,-0.075617,-0.293987,-0.333333,-0.105263,-0.067596,0.250333,-0.170455,0.498182,-0.177778,-0.202076,0.044178,0.137725,0.040650,-1.986301,0.007109,-0.023713
1,-0.146720,-0.297230,-0.617188,-0.162162,-1.728745,0.017857,-0.160976,0.051282,-1.750080,-1.762542,0.000798,-0.119565,-0.148408,-0.078532,-0.329558,-0.018373,-0.054545,-0.176471,-0.190644,-0.301870,-0.317536,-0.224299,-0.009553,-0.210419,0.087719,-0.081931,0.235441,-0.162162,-0.007322,-0.021667,-0.040541,0.111111,0.315789,0.261780,0.101740,-0.051071,0.003119,-0.045872,-0.200000,-0.200000,...,0.069884,-0.089474,-0.067568,-0.031250,-0.107944,-0.281250,-0.027972,-0.155251,-0.033333,0.128319,-0.380592,-0.171694,0.083360,0.327103,0.009524,0.028896,-0.019608,0.441878,0.198582,0.041667,-0.187050,-0.004402,0.081266,0.113208,0.033389,-0.387382,-0.071429,0.105882,-0.082370,-0.275825,-0.054795,-0.148058,-0.135135,0.065048,0.374061,-0.052632,0.039062,-0.013889,-0.136471,0.143650
2,-0.251854,-0.490700,0.918367,0.064516,0.066667,0.000000,-0.240698,0.091463,-2.333191,-0.034737,-0.196649,-1.919753,-0.093458,-0.213313,-0.053405,0.005348,-0.024038,-0.266964,-0.328899,-0.520714,-0.366658,0.072289,-0.219797,-0.322165,-0.064516,-0.169456,-0.256566,-0.064516,-0.127903,0.060477,-0.028169,0.100000,-0.210000,0.095436,0.062272,-0.717882,-0.139896,-0.048077,-0.500000,0.050000,...,0.260692,-0.052023,0.159420,0.096774,-0.273250,0.260870,0.035971,-0.081081,-0.017241,0.011765,-0.224822,-0.154062,-0.153842,-0.056338,-0.094340,0.016048,-0.060000,-0.015245,-0.242604,-0.060000,-0.207965,-0.056006,-0.002307,0.033898,0.017771,-0.188465,0.307692,-0.013298,-0.011989,0.080882,-0.173913,0.014245,0.031250,0.127036,-0.060719,0.150000,-0.037594,-0.042254,-0.068120,-0.407767
3,-0.151420,-0.064348,-0.574468,-0.030303,0.072917,0.100877,-0.350689,-0.061453,-2.000000,-0.436569,0.376862,-1.953020,0.154639,-0.026923,-0.138675,0.005319,0.123153,-0.390987,-0.161521,-0.208075,-0.248720,0.134831,-0.137280,-0.016730,0.168103,0.002558,-0.139493,0.413793,-0.011337,0.022892,0.000000,1.090909,-0.113924,0.064394,-0.014870,0.547692,-0.180723,-0.242424,-0.250000,-2.142857,...,0.172398,0.378049,-0.050000,-0.117647,0.045455,0.086207,-0.062500,-0.017647,0.122807,0.058140,-0.234471,-0.155629,-0.090936,-0.022388,0.000000,0.004442,0.265957,-0.163212,-0.242188,-0.212766,-0.173184,0.096799,-0.161616,-0.016393,-0.038095,-0.186548,0.029412,-0.056604,-0.001525,-0.027211,0.070175,-0.053371,0.060606,0.013006,-0.076287,0.009662,0.101562,-0.051471,-0.102339,-0.114754
4,-0.169942,0.046468,-0.275000,0.312500,-0.208738,0.035857,-0.108491,-0.035714,-2.000000,0.196774,-0.188604,-0.084507,0.096639,0.108366,-0.078990,0.027725,-0.017544,0.300000,-0.184109,0.235294,0.180579,0.039604,0.208899,0.160866,-0.099631,0.137480,0.247368,0.121951,-0.151438,-0.022379,-0.188406,0.434783,-0.142857,-0.007117,0.062990,-0.091451,0.051471,-0.200000,0.333333,-2.041667,...,0.171643,-0.008850,-0.157895,-1.983333,-0.042533,-0.015873,-1.988889,-0.065868,0.125000,-0.007326,0.164000,0.129412,0.000000,-0.099237,0.000000,0.14201

In [7]:
%%time

cov_hat = []

for cov in cov_matrices:
    cov_, _ = get_shrunk_covariance_matrix(cov.cov())
    break


CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 4.6 ms


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [8]:
cov_

,ACV,AMCO,AZE,BFD,BIO,BLT,BSC,BTI,CHL,CIS,CITU,CMD,CNP,CPN,DEP,EJ,FCE,HUB,KC,MOG,MRI,OHC,PDL,PLR,RA,RT,SAU,SK,SMC,TDD,UFD,VGC,WAN,WIN,WSO
ACV,0.000749,-0.000419,0.000215,-1.377628e-04,-0.000159,-0.000322,0.000044,0.000055,0.001123,-0.000230,-0.000015,-0.000433,-0.000176,-0.000002,-0.000062,0.000868,-0.000982,-0.000191,-0.000266,0.000055,-0.000324,-0.000266,-0.000272,0.000508,-0.000015,0.000577,-5.522722e-05,-0.000418,0.000193,0.000244,-0.000217,-0.000481,0.000632,0.000640,-0.000777
AMCO,-0.000419,0.019077,-0.000107,-6.746582e-04,0.000025,0.001560,-0.000221,-0.002305,-0.016404,-0.000912,0.000093,0.001155,0.000751,-0.000011,-0.000159,-0.004288,-0.000229,-0.000113,-0.000862,-0.000040,0.001055,0.000960,0.003568,-0.005583,0.000422,-0.002849,-8.030892e-04,0.000588,-0.000103,-0.003481,0.000808,0.002171,-0.000748,-0.000390,-0.002034
AZE,0.000215,-0.000107,0.029765,-8.660188e-04,-0.001842,0.003399,0.000249,-0.000289,0.005429,-0.003016,-0.000227,0.003787,-0.003521,-0.000248,-0.000662,0.004087,-0.003457,-0.001653,0.001078,-0.000498,-0.000090,-0.000891,-0.005525,0.001824,0.000660,0.001837,-6.497971e-03,-0.002503,0.000628,0.002659,-0.002182,-0.001047,0.000434,0.001448,-0.002933
BFD,-0.000138,-0.000675,-0.000866,3.313815e-03,-0.000106,-0.000823,-0.000127,-0.000727,-0.000857,-0.000617,-0.000138,-0.000476,-0.000288,0.000017,0.000155,-0.001323,0.000833,0.000322,-0.000246,-0.000246,-0.000762,-0.000839,0.000138,-0.000075,-0.000512,0.001226,4.090349e-07,0.000795,-0.000312,0.000703,-0.000138,-0.001251,-0.000789,-0.000700,0.001726
BIO,-0.000159,0.000025,-0.001842,-1.058078e-04,0.024491,0.001580,0.000042,-0.001270,0.000846,-0.001574,-0.000519,-0.000672,0.001151,-0.000032,-0.000793,0.001573,0.000516,0.001374,-0.000998,0.000495,0.001358,-0.000196,0.000731,-0.004177,-0.000191,-0.002362,-2.486259e-03,-0.001635,-0.000844,-0.001331,-0.000315,-0.000681,0.001316,-0.000363,-0.000975
BLT,-0.000322,0.001560,0.003399,-8.229446e-04,0.001580,0.020221,0.000157,-0.002008,0.001856,-0.000864,-0.000102,-0.001079,-0.000734,-0.000057,-0.001915,0.001182,-0.000400,-0.001408,-0.000201,-0.001850,0.001186,0.000853,-0.002836,-0.004023,0.000727,-0.002525,1.322813e-03,-0.003145,-0.000090,0.000624,-0.000603,0.001230,-0.001309,-0.000106,0.002595
BSC,0.000044,-0.000221,0.000249,-1.270780e-04,0.000042,0.000157,0.002898,0.001017,0.006610,0.000205,-0.000106,-0.000501,-0.001668,0.000017,0.000047,0.000737,-0.000380,-0.000287,0.000460,-0.000144,-0.000191,0.000089,-0.001153,0.000118,-0.000320,0.000625,5.120668e-05,-0.002186,-0.000467,0.000757,-0.000497,-0.000923,0.000628,0.000864,-0.000728
BTI,0.000055,-0.002305,-0.000289,-7.269680e-04,-0.001270,-0.002008,0.001017,0.013574,0.014925,0.001674,0.000066,0.000885,-0.001452,-0.000018,0.001169,0.000243,-0.000885,-0.000092,-0.001251,0.000561,-0.000468,-0.001038,-0.002045,0.000954,-0.001069,0.003643,-1.146481e-03,-0.002392,-0.001204,-0.000051,-0.000758,-0.003448,0.001022,0.002218,-0.001583
CHL,0.001123,-0.016404,0.005429,-8.574448e-04,0.000846,0.001856,0.006610,0.014925,0.546079,0.002015,-0.001068,-0.005047,-0.014921,-0.000114,0.002851,0.004759,-0.006698,0.006846,-0.006747,0.005839,0.011152,-0.009255,-0.025460,0.004866,-0.006414,0.040124,-5.459525e-03,-0.027476,-0.007684,0.003494,-0.009266,-0.022880,0.007769,0.010571,-0.010637
CIS,-0.000230,-0.000912,-0.003016,-6.172460e-04,-0.001574,-0.000864,0.000205,0.001674,0.002015,0.017984,0.000770,-0.000220,-0.000459,-0.000024,0.001607,-0.000308,-0.000219,0.000526,-0.002811,-0.000753,-0.003011,-0.000586,-0.002407,-0.002248,-0.000334,0.001224,6.134278e-04,0.001364,-0.000360,-0.002384,-0.000098,-0.002350,0.000106,-0.000980,0.001212
